In [2]:
import io
import re
import requests
import pandas as pd
import numpy as np

# Define what cruises you want

In [3]:
# Pull NES-LTER chlorophyll CSVs, apply QC, and return per-sample chl averages
CRUISES = ['AE2426', 'AR28b', 'EN608', 'EN687', 'EN695', 'EN720', 'AR61b', 
           'EN715', 'EN706', 'EN712', 'EN608', "EN661", "EN617", "EN657"]
#CRUISES = ['AR24A', 'AR24B', 'AR24C', 'AR31A', 'AR34B', 'AR38', 'AR39B', 'AR44', 'AR70B', 'AR77', 'AR88', 'AT46', 'EN617', 'EN627', 'EN644', 'EN649', 'EN655', 'EN657', 'EN661', 'EN668', 'EN706', 'EN712', 'EN715']
API_TPL = "https://nes-lter-data.whoi.edu/api/chl/{cruise}.csv"
TIMEOUT = 30

# Helper Functions

In [4]:
def fetch_chl_table(cruise: str) -> pd.DataFrame:
    """Download one cruise CSV from the API."""
    url = API_TPL.format(cruise=cruise)
    r = requests.get(url, timeout=TIMEOUT)
    r.raise_for_status()
    return pd.read_csv(io.StringIO(r.text))

def normalize_filter_size(fs: str) -> str:
    """
    Map filter_size strings to simple tokens used for averaging.
    Examples seen in MATLAB code: '>0&<200', '>5&<200', '>10&<200', '>20&<200', '>0&<10'
    Returns one of: '0','5','10','20','<10' or the original cleaned string as fallback.
    """
    s = str(fs).replace(" ", "").lower()
    # Standard forms
    if ">0" in s and ("<200" in s or "200" in s or "&" not in s):
        return "0"
    if ">5" in s and "<200" in s:
        return "5"
    if ">10" in s and "<200" in s:
        return "10plus"  # only if you later need it
    if ">20" in s and "<200" in s:
        return "20"
    if "<10" in s:
        return "<10"
    if "5to20" in s or "5-20" in s:
        return "5to20"
    # Fallbacks like '0-200' etc.
    if re.fullmatch(r"0[-–]?200", s):
        return "0"
    if re.fullmatch(r"5[-–]?200", s):
        return "5"
    if re.fullmatch(r"20[-–]?200", s):
        return "20"
    return s

def qc_chl(df: pd.DataFrame) -> pd.DataFrame:
    """Apply simple QC: remove flags 3/4, rb <= 3*blank, chl <= 0.1 µg/L."""
    out = df.copy()
    # keep only required columns if present
    keep = [c for c in ["cruise","cast","niskin","replicate","filter_size","rb","blank",
                        "chl","phaeo","quality_flag","date","longitude","latitude","depth"]
            if c in out.columns]
    out = out[keep]
    # coerce types
    for c in ["cast","niskin","rb","blank","chl","phaeo","quality_flag","longitude","latitude","depth"]:
        if c in out.columns:
            out[c] = pd.to_numeric(out[c], errors="coerce")
    if "replicate" in out.columns:
        out["replicate"] = out["replicate"].astype(str).str.strip().str.lower()
    # QC
    if "quality_flag" in out.columns:
        out = out[~out["quality_flag"].isin([3,4])]
    if {"rb","blank"} <= set(out.columns):
        out = out[out["rb"] > 3 * out["blank"]]
    if "chl" in out.columns:
        out = out[out["chl"] > 0.1]
    return out

def per_sample_meta(df: pd.DataFrame) -> pd.DataFrame:
    """Grab one copy of date/lat/lon/depth per sample for context."""
    meta_cols = [c for c in ["date","longitude","latitude","depth"] if c in df.columns]
    if not meta_cols:
        return df[["cruise","cast","niskin"]].drop_duplicates().set_index(["cruise","cast","niskin"])
    meta = (
        df[["cruise","cast","niskin"] + meta_cols]
        .sort_values(meta_cols)  # arbitrary but deterministic
        .drop_duplicates(["cruise","cast","niskin"])
        .set_index(["cruise","cast","niskin"])
    )
    return meta

def compute_replicate_means(df: pd.DataFrame) -> pd.DataFrame:
    """
    From QC data, compute per (cruise,cast,niskin) replicate-mean chl values.
    Returns columns:
      chl_0_avg, chl_5_avg, chl_10_avg, chl_20_avg (where available)
    """
    d = df.copy()
    d["filter_norm"] = d["filter_size"].map(normalize_filter_size)
    # pivot to columns by (filter_norm, replicate)
    piv = d.pivot_table(
        index=["cruise","cast","niskin"],
        columns=["filter_norm","replicate"],
        values="chl",
        aggfunc="mean"  # if multiple measurements exist
    )
    # helper to average a and b
    def mean_ab(filter_key):
        cols = []
        for rep in ["a","b"]:
            if (filter_key, rep) in piv.columns:
                cols.append((filter_key, rep))
        if not cols:
            return pd.Series(np.nan, index=piv.index)
        return piv[cols].mean(axis=1, skipna=True)

    out = pd.DataFrame(index=piv.index)
    out["chl_0_avg"]  = mean_ab("0")
    out["chl_5_avg"]  = mean_ab("5")
    out["chl_10_avg"] = mean_ab("<10")
    out["chl_20_avg"] = mean_ab("20")

    # attach one set of meta per sample
    meta = per_sample_meta(d)
    out = out.join(meta, how="left").reset_index()

    # enforce dtypes for merge keys
    out["cruise"] = out["cruise"].astype(str)
    out["cast"]   = pd.to_numeric(out["cast"], errors="coerce").astype("Int64")
    out["niskin"] = pd.to_numeric(out["niskin"], errors="coerce").astype("Int64")
    return out

# Extract Chl

In [5]:
all_chl = []
for c in CRUISES:
    try:
        df = fetch_chl_table(c)
    except Exception as e:
        print(f"Warning: failed to fetch {c}: {e}")
        continue
    df["cruise"] = df["cruise"].astype(str)
    qc = qc_chl(df)
    all_chl.append(qc)

if not all_chl:
    raise RuntimeError("No CHL data retrieved. Check cruise list or connectivity.")

chl_qc = pd.concat(all_chl, ignore_index=True)

chl_metrics = compute_replicate_means(chl_qc)

# Minimal output: one row per (cruise, cast, niskin) with total chl
chl_min = chl_metrics[["cruise","cast","niskin","chl_0_avg","date","longitude","latitude","depth"]].copy()

# Save if desired
# chl_metrics.to_csv("neslter_chl_per_sample.csv", index=False)

In [6]:
chl_metrics

,cruise,cast,niskin,chl_0_avg,chl_5_avg,chl_10_avg,chl_20_avg,date,longitude,latitude,depth
0,AR28B,1,2,1.211422,NaN,NaN,NaN,,NaN,NaN,NaN
1,AR28B,1,5,0.876879,NaN,NaN,NaN,,NaN,NaN,NaN
2,AR28B,1,8,0.844588,NaN,NaN,NaN,,NaN,NaN,NaN
3,AR28B,2,1,6.212741,NaN,NaN,NaN,2018-04-03 22:19:16+00:00,-70.883167,41.031333,39.722652
4,AR28B,2,5,4.855188,NaN,NaN,NaN,2018-04-03 22:22:32+00:00,-70.883167,41.031333,26.802747
...,...,...,...,...,...,...,...,...,...,...,...
671,EN720,20,17,0.286606,NaN,NaN,NaN,2024-09-10 20:54:35+00:00,-70.883360,40.861620,15.522000
672,EN720,20,20,0.200447,NaN,0.183725,NaN,2024-09-10 20:58:31+00:00,-70.882820,40.861410,3.939000
673,EN720,24,2,3.145140,NaN,NaN,NaN,2024-09-11 05:38:01+00:00,-70.580550,41.317980,15.633000
674,EN720,24,4,4.793017,NaN,5.369058,NaN,2024-09-11 05:39:59+00:00,-70.581560,41.317860,8.467000


In [7]:
# make cruise column uppercase
chl_metrics["cruise"] = chl_metrics["cruise"].str.upper()

# also uppercase your CRUISES list
CRUISES = [c.upper() for c in CRUISES]

chl_metrics['cruise'].unique()

array(['AR28B', 'AR61B', 'EN608', 'EN617', 'EN657', 'EN661', 'EN687',
       'EN695', 'EN706', 'EN712', 'EN715', 'EN720'], dtype=object)

# Check for missing data

In [14]:
counts = (
    chl_metrics.groupby("cruise")["chl_0_avg"]
      .agg(n_chl0="count", total="size")
)

counts = counts.reindex(CRUISES, fill_value=0).reset_index()
counts["missing"] = counts["total"] - counts["n_chl0"]
counts.rename(columns={"index": "cruise"}, inplace=True)

print(counts)

chl_path = r"C:\Users\ocron\OneDrive - Massachusetts Institute of Technology\Documents\Research\Generals\Research_Report3\Data\Chl\chl_counts.csv"
counts.to_csv(chl_path, index=False)
print("Exported to:", chl_path)

    cruise  n_chl0  total  missing
0   AE2426       0      0        0
1    AR28B      81     83        2
2    EN608      55     57        2
3    EN687      57     57        0
4    EN695      49     52        3
5    EN720      53     54        1
6    AR61B      48     53        5
7    EN715      53     54        1
8    EN706      56     58        2
9    EN712      53     56        3
10   EN608      55     57        2
11   EN661      37     38        1
12   EN617      61     62        1
13   EN657      48     52        4
Exported to: C:\Users\ocron\OneDrive - Massachusetts Institute of Technology\Documents\Research\Generals\Research_Report3\Data\Chl\chl_counts.csv


# Export data

In [15]:
out_path = r"C:\Users\ocron\OneDrive - Massachusetts Institute of Technology\Documents\Research\Generals\Research_Report3\Data\Chl\chl_conc.csv"
chl_metrics.to_csv(out_path, index=False)
print("Exported to:", out_path)

Exported to: C:\Users\ocron\OneDrive - Massachusetts Institute of Technology\Documents\Research\Generals\Research_Report3\Data\Chl\chl_conc.csv
